In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, roc_auc_score, roc_curve, 
                             precision_score, recall_score, f1_score, 
                             confusion_matrix, classification_report)
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

df = pd.read_csv('S05-hw-dataset.csv')

print("=" * 80)
print("1. ОБЩАЯ ИНФОРМАЦИЯ О ДАТАСЕТЕ")
print("=" * 80)

print(f"Размер датасета: {df.shape[0]} строк, {df.shape[1]} столбцов")

print("\nПервые 5 строк датасета:")
display(df.head())

print("\nИнформация о столбцах и типах данных:")
print(df.info())

print("\n" + "=" * 80)
print("2. БАЗОВЫЕ СТАТИСТИКИ ДЛЯ ЧИСЛОВЫХ ПРИЗНАКОВ")
print("=" * 80)

display(df.describe().T.style.background_gradient(cmap='Blues', axis=0))

print(f"\nПропущенные значения:")
missing_values = df.isnull().sum()
display(missing_values[missing_values > 0])


print("\n" + "=" * 80)
print("3. АНАЛИЗ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ (default)")
print("=" * 80)

target_distribution = df['default'].value_counts(normalize=True) * 100
print("\nРаспределение целевой переменной:")
print(f"0 (нет дефолта): {target_distribution[0]:.2f}%")
print(f"1 (дефолт): {target_distribution[1]:.2f}%")


fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].bar(['Нет дефолта (0)', 'Дефолт (1)'], target_distribution.values, color=['green', 'red'])
axes[0].set_title('Распределение целевой переменной', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Процент (%)', fontsize=12)
for i, v in enumerate(target_distribution.values):
    axes[0].text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')


axes[1].pie(target_distribution.values, labels=target_distribution.index, 
           autopct='%1.1f%%', colors=['green', 'red'], startangle=90)
axes[1].set_title('Доля классов', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('figures/target_distribution.png', dpi=300, bbox_inches='tight')
plt.show()


numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if 'client_id' in numeric_cols:
    numeric_cols.remove('client_id')

corr_subset = df[numeric_cols]

correlation_matrix = corr_subset.corr()

plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица признаков', fontsize=16, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('figures/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

target_corr = correlation_matrix['default'].sort_values(ascending=False)
print("\nКорреляция признаков с целевой переменной (default):")
display(pd.DataFrame(target_corr).style.background_gradient(cmap='RdBu_r', axis=0))

print("\n" + "=" * 80)
print("4. ПОДГОТОВКА ПРИЗНАКОВ И ТАРГЕТА")
print("=" * 80)

X = df.drop(['default', 'client_id'], axis=1)
y = df['default']

print(f"Размер матрицы признаков X: {X.shape}")
print(f"Размер вектора таргета y: {y.shape}")

print("\nТипы данных в X:")
print(X.dtypes.value_counts())

categorical_cols = X.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    print(f"\nКатегориальные признаки: {list(categorical_cols)}")
    for col in categorical_cols:
        print(f"{col}: {X[col].unique()[:10]}...")
else:
    print("\nВсе признаки являются числовыми (категориальных признаков нет)")

print("\n" + "=" * 80)
print("5. TRAIN/TEST-СПЛИТ И БЕЙЗЛАЙН-МОДЕЛЬ")
print("=" * 80)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25,        
    random_state=42,       
    stratify=y            
)

print(f"Размер обучающей выборки: {X_train.shape[0]} объектов")
print(f"Размер тестовой выборки: {X_test.shape[0]} объектов")

print(f"\nРаспределение классов в обучающей выборке:")
train_dist = y_train.value_counts(normalize=True)
print(f"0 (нет дефолта): {train_dist[0]:.2%}")
print(f"1 (дефолт): {train_dist[1]:.2%}")

print(f"\nРаспределение классов в тестовой выборке:")
test_dist = y_test.value_counts(normalize=True)
print(f"0 (нет дефолта): {test_dist[0]:.2%}")
print(f"1 (дефолт): {test_dist[1]:.2%}")


dummy_models = {
    'most_frequent': DummyClassifier(strategy='most_frequent', random_state=42),
    'stratified': DummyClassifier(strategy='stratified', random_state=42)
}

baseline_results = []

for name, model in dummy_models.items():
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_pred_proba)
    else:
        roc_auc = None
    
    accuracy = accuracy_score(y_test, y_pred)
    
    baseline_results.append({
        'Model': f'Dummy ({name})',
        'Accuracy': accuracy,
        'ROC-AUC': roc_auc,
        'Description': f'Предсказывает {name}'
    })


baseline_df = pd.DataFrame(baseline_results)
print("\nРезультаты бейзлайн-моделей:")
display(baseline_df.style.format({'Accuracy': '{:.4f}', 'ROC-AUC': '{:.4f}'})
       .background_gradient(subset=['Accuracy', 'ROC-AUC'], cmap='YlOrRd'))


best_baseline = dummy_models['stratified']
y_pred_baseline = best_baseline.predict(X_test)
y_pred_proba_baseline = best_baseline.predict_proba(X_test)[:, 1] if hasattr(best_baseline, 'predict_proba') else None

print(f"\nДля дальнейшего сравнения используем DummyClassifier(strategy='stratified')")
print(f"Accuracy лучшей бейзлайн-модели: {accuracy_score(y_test, y_pred_baseline):.4f}")


print("\n" + "=" * 80)
print("6. ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ И ПОДБОР ГИПЕРПАРАМЕТРОВ")
print("=" * 80)


logreg_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Стандартизация признаков
    ('logreg', LogisticRegression(
        max_iter=1000,             # Увеличиваем максимальное количество итераций
        random_state=42,           # Для воспроизводимости
        solver='lbfgs'             # Алгоритм оптимизации
    ))
])

print("Создан пайплайн:")
for step_name, step in logreg_pipeline.steps:
    print(f"  - {step_name}: {step}")


param_grid = {
    'logreg__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],  
    'logreg__penalty': ['l2'],  
}


grid_search = GridSearchCV(
    logreg_pipeline,
    param_grid,
    cv=5,                    # 5-кратная кросс-валидация
    scoring='roc_auc',       # Оптимизируем по ROC-AUC
    n_jobs=-1,               # Используем все доступные ядра
    verbose=1
)


print("\nНачинаем подбор гиперпараметров...")
grid_search.fit(X_train, y_train)


print("\nРезультаты GridSearchCV:")
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучший score (ROC-AUC) на кросс-валидации: {grid_search.best_score_:.4f}")


cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values('mean_test_score', ascending=False)

print("\nТоп-5 комбинаций гиперпараметров:")
top_5_results = cv_results[['params', 'mean_test_score', 'std_test_score']].head()
display(top_5_results.style.format({'mean_test_score': '{:.4f}', 'std_test_score': '{:.4f}'})
       .background_gradient(subset=['mean_test_score'], cmap='YlGn'))


c_values = []
mean_scores = []
std_scores = []

for c in param_grid['logreg__C']:
    mask = cv_results['param_logreg__C'] == c
    if mask.any():
        c_values.append(c)
        mean_scores.append(cv_results[mask]['mean_test_score'].mean())
        std_scores.append(cv_results[mask]['std_test_score'].mean())


plt.figure(figsize=(10, 6))
plt.errorbar(c_values, mean_scores, yerr=std_scores, 
             marker='o', markersize=8, capsize=5, linewidth=2)
plt.xscale('log')
plt.xlabel('Параметр регуляризации C (log scale)', fontsize=12)
plt.ylabel('ROC-AUC (кросс-валидация)', fontsize=12)
plt.title('Влияние параметра C на качество модели', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('figures/c_parameter_effect.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nОптимальное значение C: {grid_search.best_params_['logreg__C']}")


best_logreg = grid_search.best_estimator_

y_pred_logreg = best_logreg.predict(X_test)
y_pred_proba_logreg = best_logreg.predict_proba(X_test)[:, 1]

logreg_accuracy = accuracy_score(y_test, y_pred_logreg)
logreg_roc_auc = roc_auc_score(y_test, y_pred_proba_logreg)
logreg_precision = precision_score(y_test, y_pred_logreg)
logreg_recall = recall_score(y_test, y_pred_logreg)
logreg_f1 = f1_score(y_test, y_pred_logreg)

print("\nМетрики лучшей логистической регрессии на тестовой выборке:")
print(f"Accuracy: {logreg_accuracy:.4f}")
print(f"ROC-AUC: {logreg_roc_auc:.4f}")
print(f"Precision: {logreg_precision:.4f}")
print(f"Recall: {logreg_recall:.4f}")
print(f"F1-Score: {logreg_f1:.4f}")

conf_matrix = confusion_matrix(y_test, y_pred_logreg)
print("\nМатрица ошибок:")
print(conf_matrix)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Предсказано 0', 'Предсказано 1'],
            yticklabels=['Истинное 0', 'Истинное 1'])
plt.title('Матрица ошибок логистической регрессии', fontsize=14, fontweight='bold')
plt.ylabel('Истинный класс', fontsize=12)
plt.xlabel('Предсказанный класс', fontsize=12)
plt.tight_layout()
plt.savefig('figures/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nОтчет классификации:")
print(classification_report(y_test, y_pred_logreg, 
                           target_names=['Нет дефолта (0)', 'Дефолт (1)']))


fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_logreg)

if y_pred_proba_baseline is not None:
    fpr_baseline, tpr_baseline, _ = roc_curve(y_test, y_pred_proba_baseline)
    roc_auc_baseline = roc_auc_score(y_test, y_pred_proba_baseline)

plt.figure(figsize=(10, 8))

plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'Логистическая регрессия (ROC-AUC = {logreg_roc_auc:.3f})')

if y_pred_proba_baseline is not None:
    plt.plot(fpr_baseline, tpr_baseline, color='blue', lw=2, linestyle='--',
             label=f'Бейзлайн (ROC-AUC = {roc_auc_baseline:.3f})')

plt.plot([0, 1], [0, 1], color='gray', lw=1, linestyle='--', label='Случайный классификатор')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)', fontsize=12)
plt.ylabel('True Positive Rate (TPR)', fontsize=12)
plt.title('ROC-кривые классификаторов', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
plt.scatter(fpr[optimal_idx], tpr[optimal_idx], color='red', s=100, 
            label=f'Оптимальный порог = {optimal_threshold:.3f}', zorder=5)
plt.annotate(f'Порог: {optimal_threshold:.3f}', 
             xy=(fpr[optimal_idx], tpr[optimal_idx]),
             xytext=(fpr[optimal_idx] + 0.1, tpr[optimal_idx] - 0.1),
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('figures/roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nОптимальный порог классификации (ближайший к (0,1)): {optimal_threshold:.3f}")

print("\n" + "=" * 80)
print("7. СРАВНЕНИЕ МОДЕЛЕЙ И ИТОГОВЫЕ ВЫВОДЫ")
print("=" * 80)

comparison_data = [
    {
        'Модель': 'Dummy (stratified)',
        'Accuracy': accuracy_score(y_test, y_pred_baseline),
        'ROC-AUC': roc_auc_baseline if y_pred_proba_baseline is not None else 'N/A',
        'Precision': 'N/A',
        'Recall': 'N/A',
        'F1-Score': 'N/A',
        'Описание': 'Случайное предсказание с сохранением распределения классов'
    },
    {
        'Модель': 'Logistic Regression',
        'Accuracy': logreg_accuracy,
        'ROC-AUC': logreg_roc_auc,
        'Precision': logreg_precision,
        'Recall': logreg_recall,
        'F1-Score': logreg_f1,
        'Описание': f"Логистическая регрессия с C={grid_search.best_params_['logreg__C']}"
    }
]

comparison_df = pd.DataFrame(comparison_data)

print("\nСРАВНИТЕЛЬНАЯ ТАБЛИЦА МОДЕЛЕЙ")
print("-" * 100)
display(comparison_df.style.format({
    'Accuracy': '{:.4f}',
    'ROC-AUC': '{:.4f}',
    'Precision': '{:.4f}',
    'Recall': '{:.4f}',
    'F1-Score': '{:.4f}'
}).background_gradient(subset=['Accuracy', 'ROC-AUC', 'Precision', 'Recall', 'F1-Score'], 
                     cmap='YlGnBu'))

if y_pred_proba_baseline is not None:
    accuracy_improvement = (logreg_accuracy - accuracy_score(y_test, y_pred_baseline)) / accuracy_score(y_test, y_pred_baseline) * 100
    roc_auc_improvement = (logreg_roc_auc - roc_auc_baseline) / roc_auc_baseline * 100
    
    print(f"\nОтносительное улучшение по сравнению с бейзлайном:")
    print(f"Accuracy: +{accuracy_improvement:.2f}%")
    print(f"ROC-AUC: +{roc_auc_improvement:.2f}%")


fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Сравнение метрик качества моделей', fontsize=16, fontweight='bold')

metrics = ['Accuracy', 'ROC-AUC', 'Precision', 'Recall', 'F1-Score']
metric_values_logreg = [logreg_accuracy, logreg_roc_auc, logreg_precision, logreg_recall, logreg_f1]


for idx, metric in enumerate(metrics):
    ax = axes[idx // 3, idx % 3]
    
    
    values = []
    labels = []
    
    
    if metric in ['Accuracy', 'ROC-AUC']:
        if metric == 'Accuracy':
            values.append(accuracy_score(y_test, y_pred_baseline))
            labels.append('Dummy')
        elif metric == 'ROC-AUC' and y_pred_proba_baseline is not None:
            values.append(roc_auc_baseline)
            labels.append('Dummy')
    
    
    values.append(metric_values_logreg[idx])
    labels.append('LogReg')
    
    
    bars = ax.bar(labels, values, color=['skyblue', 'lightcoral'][:len(labels)])
    ax.set_title(metric, fontsize=12, fontweight='bold')
    ax.set_ylabel('Значение', fontsize=10)
    ax.set_ylim([0, 1.1])
    
    
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{value:.3f}', ha='center', va='bottom', fontsize=10)
    
    ax.grid(True, alpha=0.3, axis='y')

axes[1, 2].axis('off')

plt.tight_layout()
plt.savefig('figures/metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


logreg_model = best_logreg.named_steps['logreg']
feature_names = X.columns.tolist()


feature_importance = pd.DataFrame({
    'Признак': feature_names,
    'Коэффициент': logreg_model.coef_[0],
    'Абсолютное значение': np.abs(logreg_model.coef_[0])
})


feature_importance = feature_importance.sort_values('Абсолютное значение', ascending=False)

print("\nВАЖНОСТЬ ПРИЗНАКОВ В ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ")
print("(Отсортировано по абсолютному значению коэффициента)")
print("-" * 80)

top_n = 10
print(f"\nТоп-{top_n} самых важных признаков:")
display(feature_importance.head(top_n).style.format({'Коэффициент': '{:.4f}', 'Абсолютное значение': '{:.4f}'})
       .background_gradient(subset=['Абсолютное значение'], cmap='Reds'))

plt.figure(figsize=(12, 8))
bars = plt.barh(feature_importance.head(15)['Признак'][::-1], 
                feature_importance.head(15)['Абсолютное значение'][::-1],
                color='steelblue')

plt.xlabel('Абсолютное значение коэффициента', fontsize=12)
plt.title('Топ-15 самых важных признаков для прогнозирования дефолта', 
          fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')


for i, bar in enumerate(bars):
    width = bar.get_width()
    coeff = feature_importance.head(15).iloc[-(i+1)]['Коэффициент']
    plt.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{coeff:.3f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('figures/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "=" * 80)
print("ИТОГОВЫЙ ОТЧЕТ И ВЫВОДЫ")
print("=" * 80)

print("""
1. КРАТКОЕ РЕЗЮМЕ ЭКСПЕРИМЕНТА:
---------------------------------------------------------------
- Датасет содержал {} наблюдений с {} признаками.
- Целевая переменная (дефолт по кредиту) распределена: {}% - нет дефолта, {}% - дефолт.
- Данные были разделены на обучающую (75%) и тестовую (25%) выборки с сохранением баланса классов.
- Бейзлайн-модель (DummyClassifier) показала accuracy = {:.4f}.
- Логистическая регрессия с подбором гиперпараметров показала accuracy = {:.4f}.
""".format(
    df.shape[0], 
    X.shape[1],
    round(target_distribution[0], 1),
    round(target_distribution[1], 1),
    accuracy_score(y_test, y_pred_baseline),
    logreg_accuracy
))

print("""
2. СРАВНЕНИЕ МОДЕЛЕЙ:
---------------------------------------------------------------
- Логистическая регрессия превзошла бейзлайн-модель по всем метрикам.
- Улучшение accuracy: {:.2f}% (с {:.4f} до {:.4f}).
- ROC-AUC логистической регрессии составил {:.4f}, что указывает на хорошую разделяющую способность.
- Оптимальное значение параметра регуляризации C = {}.
""".format(
    accuracy_improvement,
    accuracy_score(y_test, y_pred_baseline),
    logreg_accuracy,
    logreg_roc_auc,
    grid_search.best_params_['logreg__C']
))

print("""
3. ВЛИЯНИЕ ПАРАМЕТРА РЕГУЛЯРИЗАЦИИ C:
---------------------------------------------------------------
- При очень маленьких значениях C (сильная регуляризация) модель недообучается.
- При очень больших значениях C (слабая регуляризация) модель может переобучаться.
- Оптимальное значение C = {} обеспечивает баланс между смещением и дисперсией.
- При этом значении модель показывает стабильное качество на кросс-валидации.
""".format(grid_search.best_params_['logreg__C']))

print("""
4. АНАЛИЗ ВАЖНЫХ ПРИЗНАКОВ:
---------------------------------------------------------------
Наиболее важными признаками для прогнозирования дефолта оказались:
1. {} (коэффициент: {:.3f})
2. {} (коэффициент: {:.3f})
3. {} (коэффициент: {:.3f})

Положительные коэффициенты увеличивают вероятность дефолта,
отрицательные коэффициенты уменьшают вероятность дефолта.
""".format(
    feature_importance.iloc[0]['Признак'], feature_importance.iloc[0]['Коэффициент'],
    feature_importance.iloc[1]['Признак'], feature_importance.iloc[1]['Коэффициент'],
    feature_importance.iloc[2]['Признак'], feature_importance.iloc[2]['Коэффициент']
))

print("""
5. ПРАКТИЧЕСКИЕ ВЫВОДЫ И РЕКОМЕНДАЦИИ:
---------------------------------------------------------------
1. Логистическая регрессия показала себя как эффективная модель для 
   прогнозирования кредитных дефолтов с accuracy {:.1%}.

2. Модель хорошо различает классы (ROC-AUC = {:.3f}), что делает её пригодной 
   для задач бинарной классификации с несбалансированными данными.

3. Для улучшения модели можно рассмотреть:
   - Добавление взаимодействий признаков
   - Использование более сложных методов обработки несбалансированности
   - Применение других алгоритмов (Random Forest, Gradient Boosting) для сравнения

4. При внедрении в производство рекомендуется:
   - Регулярно переобучать модель на новых данных
   - Мониторить смещение данных (data drift)
   - Использовать калибровку вероятностей для получения реалистичных оценок
""".format(logreg_accuracy, logreg_roc_auc))

print("\n" + "=" * 80)
print("ЭКСПЕРИМЕНТ УСПЕШНО ЗАВЕРШЕН")
print("=" * 80)

import joblib
import os


os.makedirs('artifacts', exist_ok=True)


joblib.dump(best_logreg, 'artifacts/best_logreg_model.pkl')
print("Лучшая модель сохранена в файл: 'artifacts/best_logreg_model.pkl'")


results_summary = {
    'best_params': grid_search.best_params_,
    'test_accuracy': logreg_accuracy,
    'test_roc_auc': logreg_roc_auc,
    'test_precision': logreg_precision,
    'test_recall': logreg_recall,
    'test_f1': logreg_f1,
    'feature_importance': feature_importance.to_dict(),
    'comparison_results': comparison_df.to_dict()
}

import json
with open('artifacts/experiment_results.json', 'w') as f:
    json.dump(results_summary, f, indent=4, default=str)

print("Результаты эксперимента сохранены в файл: 'artifacts/experiment_results.json'")

# Сохраняем финальную таблицу сравнения
comparison_df.to_csv('artifacts/models_comparison.csv', index=False, encoding='utf-8')
print("Таблица сравнения моделей сохранена в файл: 'artifacts/models_comparison.csv'")


SyntaxError: unterminated string literal (detected at line 740) (1324575026.py, line 740)